In [15]:
import os
import omnikeeper_client as okc
from omnikeeper_client import TraitDefinition, TraitAttributeDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER
from dotenv import load_dotenv

In [3]:
load_dotenv('../.env.sample')

True

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
trait_name = "python_client_demo.test"
layer_name = "testlayer"

create a omnikeeper client

In [6]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

ensure trait exists

In [ ]:
okc.upsert_trait(okapiclient, TraitDefinition(trait_name, [
        TraitAttributeDefinition("id", "test.id", ATTRIBUTETYPE_INTEGER),
        TraitAttributeDefinition("array", "test.array", ATTRIBUTETYPE_TEXT, is_array=True),
        TraitAttributeDefinition("some_key", "test.mapped_some_key", ATTRIBUTETYPE_TEXT),
      ]))

ensure layer exists

In [ ]:
okc.create_layer(okapiclient, layer_name)

Create the input data in format of **input : List[Dict[str,Any]]**

In [7]:
initial_data = [
    {"id": 1, "array": ["a", "b"], "some_key": "Value 1"},
    {"id": 3, "array": ["c", "d"], "some_key": "another Value"},
]
initial_data

[{'id': 1, 'array': ['a', 'b'], 'some_key': 'Value 1'},
 {'id': 3, 'array': ['c', 'd'], 'some_key': 'another Value'}]

write data to layer ``testlayer``

In [10]:
# simple_traits.bulk_replace(client, trait_name=trait_name, input=df_init, id_attributes=["id"], id_relations=[], write_layer=layer_name, filter={})

okc.bulk_replace_trait_entities_by_filter(okapiclient, trait_name=trait_name, input=initial_data, id_attributes=["id"], id_relations=[], write_layer=layer_name, filter={})

True

load data again from omnikeeper

In [11]:
# df_work = simple_traits.get_all(client, trait_name=trait_name, layers=[layer_name])

updated_data = okc.get_all_traitentities(okapiclient, trait_name=trait_name, layers=[layer_name])
updated_data

[{'ciid': '24cad39a-25ad-4bfb-b00d-28e4ff48a633',
  'id': 1,
  'array': ['a', 'b'],
  'some_key': 'Value 1'},
 {'ciid': '5a91f180-f24f-4ed5-a9e8-8fa9d5b564bc',
  'id': 3,
  'array': ['c', 'd'],
  'some_key': 'another Value'}]

modify the fetched traitentities

In [12]:
# change things in the returned data
# change single value of existing data
updated_data[0]["array"] = ["a", "b", "z"]

# add a new row, using our own created CIID
updated_data.append({"ciid": str(uuid.uuid4()), "id": 4, "array": ["x"], "some_key": "the new guy"})

# drop a row
updated_data.pop(1)
updated_data

[{'ciid': '24cad39a-25ad-4bfb-b00d-28e4ff48a633',
  'id': 1,
  'array': ['a', 'b', 'z'],
  'some_key': 'Value 1'},
 {'ciid': '79a734d4-2ce4-4c6e-b0c2-c5fb91597b97',
  'id': 4,
  'array': ['x'],
  'some_key': 'the new guy'}]

write it back

In [13]:
# simple_traits.set_all(client, trait_name=trait_name, input=df_work, write_layer=layer_name)

okc.bulk_replace_trait_entities(okapiclient, trait_name=trait_name, input=updated_data, write_layer=layer_name)

True

Fetch again the data

In [14]:
# df_final = simple_traits.get_all(client, trait_name=trait_name, layers=[layer_name])

final_data = okc.get_all_traitentities(okapiclient, trait_name=trait_name, layers=[layer_name])
final_data

[{'ciid': '24cad39a-25ad-4bfb-b00d-28e4ff48a633',
  'id': 1,
  'array': ['a', 'b', 'z'],
  'some_key': 'Value 1'},
 {'ciid': '79a734d4-2ce4-4c6e-b0c2-c5fb91597b97',
  'id': 4,
  'array': ['x'],
  'some_key': 'the new guy'}]